In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("kaggle_sales/train.csv")
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
from sklearn.model_selection import train_test_split
X = data.drop('SalePrice', axis=1)
y = data.SalePrice
X_train, X_val ,y_train, y_val = train_test_split(X,y, random_state=16)

In [4]:
X_train.shape

(1095, 80)

In [5]:
X_train.isnull().sum().sort_values()

Id                0
BsmtUnfSF         0
TotalBsmtSF       0
SaleType          0
HeatingQC         0
               ... 
FireplaceQu     522
Fence           874
Alley          1028
MiscFeature    1053
PoolQC         1091
Length: 80, dtype: int64

In [6]:
columns_dropped = list(X_train.loc[:,X_train.isnull().sum()>100].columns)
columns_dropped

['LotFrontage', 'Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

In [7]:
#Lot frontage and alley are included in totat lot.
#fire place is innecesary

In [8]:
#dropping columns with lot of NA values
X_train_dropped = X_train.drop(columns_dropped,axis=1)

In [9]:
X_train_dropped.isnull().sum().sort_values()

Id               0
SaleType         0
Heating          0
HeatingQC        0
CentralAir       0
                ..
GarageYrBlt     57
GarageQual      57
GarageCond      57
GarageType      57
GarageFinish    57
Length: 74, dtype: int64

It would be better to summarise all the Garage features into a single one
From data desc, 'GarageCars: Size of garage in car capacity' is best, describing both has_garage and its size.
Garage condition remains in doubt
'GarageCond'

In [10]:
X_train_dropped = X_train_dropped.drop(['GarageType','GarageYrBlt','GarageFinish','GarageArea','GarageCond'], axis=1)

In [11]:
cat_cols = [cname for cname in X_train_dropped.columns if
                    X_train_dropped[cname].nunique() < 10 and 
                    X_train_dropped[cname].dtype == "object"]

num_cols = [cname for cname in X_train_dropped.columns if 
                X_train_dropped[cname].dtype in ['int64', 'float64']]

Very few samples have different 'GarageCond' and 'GarageQual', so we will use only one

In [12]:
X_train_dropped[cat_cols]

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,BldgType,...,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,GarageQual,PavedDrive,SaleType,SaleCondition
678,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,...,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Y,New,Partial
1340,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,...,GasA,TA,Y,SBrkr,TA,Typ,TA,Y,WD,Normal
1075,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,...,GasA,TA,Y,SBrkr,TA,Typ,TA,Y,CWD,Normal
1130,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,...,GasA,TA,Y,SBrkr,Gd,Min2,TA,Y,WD,Normal
1210,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,...,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Y,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,RM,Pave,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,...,GasA,Gd,Y,FuseA,Fa,Typ,TA,Y,WD,Normal
581,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,...,GasA,Ex,Y,SBrkr,Ex,Typ,TA,Y,New,Partial
121,RM,Pave,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,...,GasA,Gd,Y,SBrkr,TA,Typ,TA,P,WD,Normal
1262,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,...,GasA,Ex,Y,FuseA,TA,Min2,TA,Y,WD,Normal


In [13]:
X_train_dropped[cat_cols].nunique().sort_values()

Street           2
Utilities        2
CentralAir       2
PavedDrive       3
LandSlope        3
BsmtCond         3
KitchenQual      4
ExterQual        4
MasVnrType       4
BsmtExposure     4
BsmtQual         4
LandContour      4
LotShape         4
Electrical       4
HeatingQC        5
GarageQual       5
MSZoning         5
ExterCond        5
BldgType         5
LotConfig        5
BsmtFinType1     6
BsmtFinType2     6
Heating          6
RoofStyle        6
Foundation       6
SaleCondition    6
RoofMatl         7
Condition2       7
Functional       7
HouseStyle       8
SaleType         9
Condition1       9
dtype: int64

In [14]:
#doesn't seem to be a num feature being considered a categorical one
#let's ordinal encode the features meant from the data_description to have a numeric order, and one hot encode the rest

In [15]:
#avoiding dirty data in test or new instances
for col in cat_cols:
    X_train_dropped[col] = X_train_dropped[col].str.lower()
X_train_dropped[cat_cols]

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,BldgType,...,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,GarageQual,PavedDrive,SaleType,SaleCondition
678,rl,pave,ir1,lvl,allpub,inside,gtl,norm,norm,1fam,...,gasa,ex,y,sbrkr,gd,typ,ta,y,new,partial
1340,rl,pave,reg,lvl,allpub,inside,gtl,norm,norm,1fam,...,gasa,ta,y,sbrkr,ta,typ,ta,y,wd,normal
1075,rl,pave,reg,lvl,allpub,inside,gtl,norm,norm,1fam,...,gasa,ta,y,sbrkr,ta,typ,ta,y,cwd,normal
1130,rl,pave,reg,lvl,allpub,inside,gtl,norm,norm,1fam,...,gasa,ta,y,sbrkr,gd,min2,ta,y,wd,normal
1210,rl,pave,reg,lvl,allpub,inside,gtl,norm,norm,1fam,...,gasa,ex,y,sbrkr,gd,typ,ta,y,wd,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,rm,pave,reg,lvl,allpub,inside,gtl,norm,norm,1fam,...,gasa,gd,y,fusea,fa,typ,ta,y,wd,normal
581,rl,pave,reg,lvl,allpub,inside,gtl,norm,norm,1fam,...,gasa,ex,y,sbrkr,ex,typ,ta,y,new,partial
121,rm,pave,reg,lvl,allpub,inside,gtl,norm,norm,1fam,...,gasa,gd,y,sbrkr,ta,typ,ta,p,wd,normal
1262,rl,pave,reg,lvl,allpub,inside,gtl,norm,norm,1fam,...,gasa,ex,y,fusea,ta,min2,ta,y,wd,normal


In [16]:
# BsmtExposure FireplaceQu
#not : poolqc FireplaceQu', 'GarageCond'
qc_order = {"na":0,"po":1, "fa":2, "ta":3,'gd':4,'ex':5}
col_to_ordinal = ['ExterQual','ExterCond','BsmtQual','BsmtCond','HeatingQC','KitchenQual','GarageQual']
X_train_dropped[col_to_ordinal] = X_train_dropped[col_to_ordinal].replace(qc_order)

In [17]:
catcol_wout_ordinal = cat_cols
for col in col_to_ordinal:
    if col in catcol_wout_ordinal:
        catcol_wout_ordinal.remove(col)

In [18]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse= False,handle_unknown='ignore')

In [19]:
ohed_col = pd.DataFrame(ohe.fit_transform(X_train_dropped[catcol_wout_ordinal]))
ohed_col.index = X_train_dropped.index
X_train_dropped = X_train_dropped.drop(catcol_wout_ordinal, axis=1)
concat_X_train_dropped = pd.concat([X_train_dropped, ohed_col], axis=1)

In [20]:
concat_X_train_dropped

,Id,MSSubClass,LotArea,Neighborhood,OverallQual,OverallCond,YearBuilt,YearRemodAdd,Exterior1st,Exterior2nd,...,125,126,127,128,129,130,131,132,133,134
678,679,20,11844,StoneBr,8,5,2008,2008,VinylSd,VinylSd,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1340,1341,20,8294,NAmes,4,5,1971,1971,MetalSd,MetalSd,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1075,1076,70,13125,Crawfor,7,6,1940,1984,Wd Sdng,Wd Sdng,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1130,1131,50,7804,SWISU,4,3,1928,1950,WdShing,Plywood,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1210,1211,60,11218,SawyerW,6,5,1992,1992,Wd Sdng,Wd Sdng,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,1346,30,6000,OldTown,4,4,1920,1950,MetalSd,MetalSd,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
581,582,20,12704,NridgHt,8,5,2008,2009,VinylSd,VinylSd,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
121,122,50,6060,IDOTRR,4,5,1939,1950,AsbShng,AsbShng,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1262,1263,50,11250,ClearCr,4,5,1957,1989,Wd Sdng,Wd Sdng,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [21]:
#now dealing with num data. seems there are some dates, and we also have to remove neighbourhoods and other cat data that remained

In [22]:
cat_cols2 = [cname for cname in X_train_dropped.columns if
                    X_train_dropped[cname].dtype == "object"]
cat_cols2

['Neighborhood', 'Exterior1st', 'Exterior2nd']

In [23]:
X_train_edit = concat_X_train_dropped.drop(cat_cols2 ,axis=1)
X_train_edit = X_train_edit.drop(['YearRemodAdd','YearBuilt','MoSold', 'YrSold'] ,axis=1)
X_train_edit

,Id,MSSubClass,LotArea,OverallQual,OverallCond,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,...,125,126,127,128,129,130,131,132,133,134
678,679,20,11844,8,5,464.0,4,3,5.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1340,1341,20,8294,4,5,0.0,3,3,3.0,3.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1075,1076,70,13125,7,6,0.0,3,3,3.0,3.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1130,1131,50,7804,4,3,0.0,3,3,3.0,3.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1210,1211,60,11218,6,5,0.0,4,3,4.0,3.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,1346,30,6000,4,4,0.0,3,3,3.0,3.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
581,582,20,12704,8,5,306.0,5,3,5.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
121,122,50,6060,4,5,0.0,3,3,3.0,3.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1262,1263,50,11250,4,5,0.0,3,3,3.0,3.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [24]:
from sklearn.impute import SimpleImputer
X_train_edit = X_train_edit.set_index('Id')
num_cols = [cname for cname in X_train_edit.columns if 
                X_train_edit[cname].dtype in ['int64', 'float64']]

imputer = SimpleImputer()
print(X_train_edit[num_cols])
imputed_X_train_edit = pd.DataFrame( imputer.fit_transform(X_train_edit[num_cols]))
imputed_X_train_edit.columns = X_train_edit.columns

      MSSubClass  LotArea  OverallQual  OverallCond  MasVnrArea  ExterQual  \
Id                                                                           
679           20    11844            8            5       464.0          4   
1341          20     8294            4            5         0.0          3   
1076          70    13125            7            6         0.0          3   
1131          50     7804            4            3         0.0          3   
1211          60    11218            6            5         0.0          4   
...          ...      ...          ...          ...         ...        ...   
1346          30     6000            4            4         0.0          3   
582           20    12704            8            5       306.0          5   
122           50     6060            4            5         0.0          3   
1263          50    11250            4            5         0.0          3   
682           50     4500            5            5         0.0 

C:\Users\zubik\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\zubik\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [25]:
#All ready for model validation!

In [26]:
imputed_X_train_edit

,MSSubClass,LotArea,OverallQual,OverallCond,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtFinSF1,...,125,126,127,128,129,130,131,132,133,134
0,20.0,11844.0,8.0,5.0,464.0,4.0,3.0,5.0,3.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,20.0,8294.0,4.0,5.0,0.0,3.0,3.0,3.0,3.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,70.0,13125.0,7.0,6.0,0.0,3.0,3.0,3.0,3.0,410.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,50.0,7804.0,4.0,3.0,0.0,3.0,3.0,3.0,3.0,622.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,11218.0,6.0,5.0,0.0,4.0,3.0,4.0,3.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,30.0,6000.0,4.0,4.0,0.0,3.0,3.0,3.0,3.0,250.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1091,20.0,12704.0,8.0,5.0,306.0,5.0,3.0,5.0,3.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1092,50.0,6060.0,4.0,5.0,0.0,3.0,3.0,3.0,3.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1093,50.0,11250.0,4.0,5.0,0.0,3.0,3.0,3.0,3.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [27]:
#for X
def pipeline(data):
    columns_dropped = ['LotFrontage', 'Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']
    data = data.drop(columns_dropped,axis=1)
    data = data.drop(['GarageType','GarageYrBlt','GarageFinish','GarageArea','GarageCond'], axis=1)
    cat_col = ['MSZoning','Street','LotShape',
     'LandContour','Utilities','LotConfig','LandSlope','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','MasVnrType','ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1',
     'BsmtFinType2',
     'Heating',
     'HeatingQC',
     'CentralAir',
     'Electrical',
     'KitchenQual',
     'Functional',
     'GarageQual',
     'PavedDrive',
     'SaleType',
     'SaleCondition']
    for col in cat_col:
        data[col] = data[col].str.lower()
        
    qc_order = {"na":0,"po":1, "fa":2, "ta":3,'gd':4,'ex':5}
    col_to_ordinal = ['ExterQual','ExterCond','BsmtQual','BsmtCond','HeatingQC','KitchenQual','GarageQual']
    data[col_to_ordinal] = data[col_to_ordinal].fillna("na")  #ACA
    data[col_to_ordinal] = data[col_to_ordinal].replace(qc_order)
    
    catcol_wout_ordinal = cat_cols
    for col in col_to_ordinal:
        if col in catcol_wout_ordinal:
            catcol_wout_ordinal.remove(col)
            
    ohed_col = pd.DataFrame(ohe.transform(data[catcol_wout_ordinal]))
    ohed_col.index = data.index
    data = data.drop(catcol_wout_ordinal, axis=1)
    data = pd.concat([data, ohed_col], axis=1)
    cat_cols2 = ['Neighborhood', 'Exterior1st', 'Exterior2nd']
    data = data.drop(cat_cols2 ,axis=1)
    data = data.drop(['YearRemodAdd','YearBuilt','MoSold', 'YrSold'] ,axis=1)
    num_cols = [cname for cname in data.columns if 
                data[cname].dtype in ['int64', 'float64']]
    data = data.set_index('Id')

    imputed_data = pd.DataFrame( imputer.transform(data[['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'MasVnrArea', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageQual', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134]
]))
    imputed_data.columns = data.columns
    imputed_data.columns = imputed_data.columns.astype(str)
    return imputed_data

In [28]:
from sklearn.linear_model import LinearRegression 
linear_model = LinearRegression()
linear_model.fit(imputed_X_train_edit, y_train)
X_val_edit = pipeline(X_val)
# preds = .predict(X_val_edit)
# print(preds)
# print(y_val)
linear_model.score(X_val_edit, y_val)

C:\Users\zubik\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\zubik\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\zubik\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


0.5935852150731434

In [29]:
from sklearn.model_selection import cross_val_score

full_X_edit = pipeline(X)
scores_linear =  cross_val_score(LinearRegression(), full_X_edit, y)
scores_linear.mean()

C:\Users\zubik\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


0.6279377972448067

In [30]:
from sklearn.ensemble import RandomForestRegressor
scores_forest =  cross_val_score(RandomForestRegressor(), full_X_edit, y)
scores_forest.mean()
forest_model = RandomForestRegressor().fit(full_X_edit, y)

In [33]:
X_test = pd.read_csv("kaggle_sales/test.csv")
X_test_edit = pipeline(X_test)
preds = forest_model.predict(X_test_edit)

test_id = X_test['Id']

output = pd.DataFrame({'Id': test_id,
                       'SalePrice': preds})
output.to_csv('./submission.csv', index=False)

C:\Users\zubik\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
